# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [85]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [86]:
# Import Clean City Data
weather = os.path.join('..', 'WheaterPy','Output_Data','Clean_City_Data.csv')
weather_df = pd.read_csv(weather)
weather_df

,City,Lat,Lng,Max Temp (C),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,Clyde River,70.4692,-68.5914,-0.87,86,40,7.72,CA,1622047223
1,Lebu,-37.6167,-73.6500,15.92,81,98,3.63,CL,1622047165
2,Busselton,-33.6500,115.3333,11.69,64,100,5.46,AU,1622047242
3,Rikitea,-23.1203,-134.9692,24.84,79,38,11.23,PF,1622047221
4,Alofi,-19.0595,-169.9187,23.94,94,88,1.54,NU,1622047344
...,...,...,...,...,...,...,...,...,...
562,Jaciara,-15.9653,-54.9683,33.28,39,6,0.73,BR,1622047966
563,Verkhnevilyuysk,63.4453,120.3167,6.32,88,91,1.69,RU,1622047966
564,Lahaina,20.8783,-156.6825,23.03,83,36,3.10,US,1622047967
565,Superior,46.7208,-92.1041,10.67,55,75,7.72,US,1622047967


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [88]:
# Store latitude and longitude in locations
world_location = weather_df[['Lat', 'Lng']]

# Store humidity
humidity = weather_df['Humidity (%)']

In [89]:
# Plot Heatmap
# Customize the size of the figure
figure_layout = {
    'width': '1000px',
    'height': '700px',
    'border': '1px solid black',
    'padding': '3px',
    'margin': '0 auto 0 auto'}

fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(world_location, weights=humidity, 
                                 dissipating=False, max_intensity=1,
                                 point_radius=0.5)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [90]:
fitting_weather = weather_df.loc[(weather_df['Max Temp (C)']>=15) & (weather_df['Max Temp (C)']<=35) & (weather_df['Wind Speed (mph)']<=30) & (weather_df['Cloudiness (%)'] == 0),:]
fitting_weather.reset_index(drop=True)

,City,Lat,Lng,Max Temp (C),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,Cape Town,-33.9258,18.4232,15.07,74,0,1.08,ZA,1622047711
1,Terenos,-20.4422,-54.8603,30.84,45,0,2.53,BR,1622047832
2,Miran Shah,33.0086,70.0697,29.48,30,0,1.18,PK,1622047833
3,Port Elizabeth,-33.9180,25.5701,16.16,77,0,1.54,ZA,1622047834
4,Abu Kamal,34.4506,40.9171,33.32,13,0,3.06,SY,1622047834
5,Abhā,18.2164,42.5053,23.14,60,0,1.97,SA,1622047373
6,East London,-33.0153,27.9116,17.52,88,0,2.06,ZA,1622047182
7,Santa Fe,-31.6333,-60.7000,22.22,41,0,1.79,AR,1622047574
8,Mar del Plata,-38.0023,-57.5575,15.91,59,0,7.60,AR,1622047755
9,Abu Samrah,35.3029,37.1841,28.43,22,0,5.44,SY,1622047849


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [101]:
hotel = []
target_search = 'hotel'
radius = 8000

for city in range(len(fitting_weather['City'])):
    lat = fitting_weather.loc[city]['Lat']
    lng = fitting_weather.loc[city]['Lng']
    coords = f'{lat},{lng}'
    params = {
    "location": coords,
    "keyword": target_search,
    "radius": radius,
    "key": gkey}
    
    b_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    h_response = requests.get(base_url, params)
    
    try:
        hotel.append(h_response['request'][0]['name'])
    except:
        hotel.append('No hotels nearby')


KeyError: 0

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
